# Be able to investigate client requirements for data analysis
# 2.4 Quantitative data analysis

* mean
* median
* standard deviation
* range

## Set up

In [2]:
try:
    import micropip
    await micropip.install(["pyoliteutils", "textblob"])
except:
    pass

In [3]:
from pyoliteutils import *
import pandas as pd

## Crowd / Attendance Data

In [4]:
crowds = pd.read_csv("../data/eagles/crowds.csv")
## https://datascienceparichay.com/article/pandas-extract-year-from-datetime-column/
crowds

,Date,Opponents,Attendance
0,02/09/1984,Rochdale,1425
1,09/09/1984,Fulham,1145
2,23/09/1984,Salford,1159
3,14/10/1984,Runcorn,1076
4,28/10/1984,Bridgend,826
...,...,...,...
583,24/07/2022,Workington,644
584,07/08/2022,Whitehaven,727
585,12/08/2022,Halifax,569
586,02/09/2022,Featherstone,912


## Get Usable Date Information

In [5]:
crowds['Date'] = pd.to_datetime(crowds['Date'], format="%d/%m/%Y")
crowds['Year'] = crowds['Date'].dt.year
crowds['month'] = crowds['Date'].dt.month
crowds['month_name'] = crowds['Date'].dt.month_name()
crowds

,Date,Opponents,Attendance,Year,month,month_name
0,1984-09-02,Rochdale,1425,1984,9,September
1,1984-09-09,Fulham,1145,1984,9,September
2,1984-09-23,Salford,1159,1984,9,September
3,1984-10-14,Runcorn,1076,1984,10,October
4,1984-10-28,Bridgend,826,1984,10,October
...,...,...,...,...,...,...
583,2022-07-24,Workington,644,2022,7,July
584,2022-08-07,Whitehaven,727,2022,8,August
585,2022-08-12,Halifax,569,2022,8,August
586,2022-09-02,Featherstone,912,2022,9,September


## Saving for later

In [7]:
crowds.to_csv("../data/eagles/crowds_clean.csv")

## Other useful Opponent / Match Data?

### Location of opponents, and distance to Sheffield

#### Possible Data Sources : 
- https://api.postcodes.io/places?q=[query]
- Nominatum / GeoPy
    - https://geopy.readthedocs.io/en/stable/
    - https://nominatim.org/release-docs/latest/api/Search/ 
    - https://medium.com/@gopesh3652/geocoding-with-python-using-nominatim-a-beginners-guide-220b250ca48d 



### Weather in Sheffield

Could do daily weather at the correct stadium location but monthly for Sheffield's Weather station will be easier at first

#### Possible Data Sources : 

- https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data saved as [text](../data/eagles/sheffield_montly_weather.csv)
- https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=39.099724&lon=-94.578331&dt=1643803200&appid=b112bddb3ca3876644b7c695768ae96d

In [6]:
sheffield_monthly_weather = pd.read_csv("../data/eagles/sheffield_monthly_weather.csv")
sheffield_monthly_weather

,yyyy,mm,tmax degC,tmin degC,af days,rain mm,sun hours
0,1883,1,6.3,1.7,6,122.1,---
1,1883,2,8,2.8,2,69.8,---
2,1883,3,4.8,-1.6,23,29.6,---
3,1883,4,12.2,3.8,2,74,---
4,1883,5,14.7,6.2,0,31.2,---
...,...,...,...,...,...,...,...
1689,2023,10,14.9,8.9,0,203.2,77.5# Provisional
1690,2023,11,9.4,5,2,108.2,68.1# Provisional
1691,2023,12,9.1,* 4.7,5,174.3,* 31.5# Provisional
1692,2024,1,7,2.6,6,72.4,54.9# Provisional


## Stadium Data

In [9]:
stadiums = pd.read_csv("../data/eagles/stadiums.csv")
stadiums

,Stadium,Postcode
0,OWLERTON,S6 2DE
1,MILLMOOR,S60 1HB
2,HEADINGLEY,LS6 3BR
3,BRAMALL LANE,S2 4QX
4,HILLSBOROUGH,S6 1SW
5,OAKWELL,S71 1ET
6,BELLE VUE,DN4 5DX
7,SALTERGATE,S40 4SX
8,TATTERSFIELD,DN4 5JW
9,THRUM HALL,HX1 4LG


## Adding the Location

In [10]:
import postcodes_io_api
api  = postcodes_io_api.Api()

def get_latlong(postcode):
    latitude = None
    longitude = None
    
    data = api.get_postcode(postcode)
    #print("data", data)
    if data["status"] != 200 :
        data = api.get_outcode(postcode)
    
    if data["status"] == 200 :        
        if "latitude" in data["result"]:
            latitude = data["result"]["latitude"]
            longitude = data["result"]["longitude"]
    #print("latitude, longitude", latitude, longitude)
    return latitude, longitude
    

def get_latlongs(df):
    if ("Latitude" not in df) and ("Postcode" in df):    
      try:
        df[["Latitude", "Longitude"]] = df.apply(
            lambda row: get_latlong(row["Postcode"]), axis=1, result_type="expand"
        )
      except Exception as e:
          print('Postcode Conversion failed : '+ str(e))
    return df

In [11]:
stadiums = get_latlongs(stadiums)
stadiums


,Stadium,Postcode,Latitude,Longitude
0,OWLERTON,S6 2DE,53.406031,-1.493303
1,MILLMOOR,S60 1HB,53.428758,-1.369397
2,HEADINGLEY,LS6 3BR,53.816081,-1.580617
3,BRAMALL LANE,S2 4QX,53.371341,-1.469862
4,HILLSBOROUGH,S6 1SW,53.410844,-1.500859
5,OAKWELL,S71 1ET,53.552266,-1.468631
6,BELLE VUE,DN4 5DX,53.517626,-1.108750
7,SALTERGATE,S40 4SX,53.238963,-1.434745
8,TATTERSFIELD,DN4 5JW,53.509088,-1.113845
9,THRUM HALL,HX1 4LG,53.721864,-1.884001


In [12]:
stadiums.to_csv("../data/eagles/stadiums_with_latlong.csv")

## Survey Data

In [14]:
questionnaire = pd.read_csv("../data/eagles/questionnaire.csv")
questionnaire

,Timestamp,Are you:,When did you first attend a Sheffield Eagles game?,Would you class yourself as:,When did you last attend a game (home or away)?,What made you first start watching the Sheffield Eagles?,What is your favourite memory of the Sheffield Eagles?,"Do you have any Sheffield Eagles memorabilia - old photographs, match day programmes, stickers, shirts, press cuttings etc - that you'd be willing to share with the project? If so please provide brief details.","Can you quickly describe what the Sheffield Eagles means to you, this could be because you are a supporter, a player, a sponsor or an organisation that works with the Club.",What are some of the stories that are part of the history of the Sheffield Eagles that you think deserve sharing to a wider audience?,...,"What do you see as the possible benefits of a project like this, for you personally, the organisation you work for, rugby league in general or the club specifically?",Do you think there is a need for a project like this? If so could you say what that need is.,Please let us have any other comments about the project.,Are You?,How old are you?,How would you describe yourself,Could you provide your postcode? This will help us understand where people are responding from.,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,9/1/2021 14:05:26,A player or member of club staff,2011- 2021,Someone who attends games home or away on a re...,29 August 2021,"Student in Sheffield, was a St. Helens fan and...",Winning the 1895 Cup,No,"My life really! Spend most of my time, both wo...","1895 Cup Win, The Supporters helping save the ...",...,"Reaching a wider audience, gaining better 'cut...",Absolutely a club that has a short but rich hi...,NaN,Male,25-39,White,S70,NaN,NaN,NaN
1,9/2/2021 9:23:44,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who has a season ticket and attends al...,Last Sunday away at London,I had already been watching the game for a few...,Challenge Cup win in 98,Yes lots of shirts.,The club has been a major part of my life for ...,It would be good for some of the newer fans to...,...,It will educate people thinking of supporting ...,To give the club a higher profile in the city ...,"No, I think everything has been covered.",Male,50-59,White,S2,NaN,NaN,NaN
2,9/2/2021 9:25:42,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Win over Oldham at home,Something the whole family could do together,Winning league for the second time,Lots of photos,Togetherness & passion,Endurance with little support from Council and...,...,Ability to show what the Eagles are about and ...,Yes to preserve the history for future and to ...,NaN,Male,50-59,White,S2,NaN,NaN,NaN
3,9/2/2021 10:05:39,A current Sheffield Eagles fan who attends games,1984 - 1990,Someone who attends games home or away on a re...,Batley Away match,1988,Challenge cup final 1998,"Got replica shirts , Programmes , Signed shirt",local rugby league Club,NaN,...,NaN,"Yes , Every club should have a heritage site",NaN,Male,50-59,White,S70,NaN,NaN,NaN
4,9/2/2021 10:53:30,A current Sheffield Eagles fan who attends games,1991 - 2000,Someone who has a season ticket and attends al...,Two weeks ago - Batley away.,Mum and Dad were big fans.,Back-to-back Grand Final wins.,Nothing from the past.,The Eagles mean a lot to me as I've met some a...,The years between 2014-2017 where we had a dif...,...,It uncovers new stories that make Rugby League...,Yes I think there is a need as there will be p...,NaN,Male,16-25,White,S5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,9/27/2021 8:00:25,A player or member of club staff,NaN,A current or former player,NaN,NaN,Winning the 2nd division final at Old Trafford,Yes,A brilliant closenit club,NaN,...,NaN,NaN,NaN,Male,60-69,White,WF10,NaN,NaN,NaN
123,9/27/2021 10:13:35,A Sheffield Eagles fan who no longer attends g...,1991 - 2000,A current or former player,2014,"Fat

In [ ]:
questionnaire_quantitive = questionnaire[[
    questionnaire.columns[1],
    questionnaire.columns[2],
    questionnaire.columns[3],
    questionnaire.columns[14],
    questionnaire.columns[15],
    questionnaire.columns[16],
    questionnaire.columns[17]
]]
questionnaire_quantitive

In [ ]:

questionnaire_quantitive = questionnaire_quantitive.rename(columns={
    'Could you provide your postcode? This will help us understand where people are responding from. ':"Postcode",
},inplace=True)
questionnaire_quantitive

In [ ]:
questionnaire_quantitive = get_latlongs(questionnaire_quantitive)
questionnaire_quantitive

In [ ]:
import folium 
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
start_coords = (46.9540700, 142.7360300);
    
m = folium.Map(
    location=start_coords, 
    zoom_start=14, 
    #tiles="Stamen Terrain",
    width="100%", height="100%",
    control_scale=True,
);

only_located = questionnaire[["Are you:","Latitude", "Longitude"]].dropna()


only_located


In [ ]:
only_located_fans = only_located[only_located["Are you:"].str.contains("A current Sheffield Eagles fan who attends games")]

In [ ]:

heat_data = [[row["Latitude"],row["Longitude"]] for index, row in only_located_fans.iterrows() if "Latitude" in row and "Longitude" in row]

HeatMap(heat_data).add_to(m)


m.fit_bounds(m.get_bounds(), padding=(30, 30))

folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('openstreetmap').add_to(m)

# Add the option to switch tiles
folium.LayerControl().add_to(m)

fs = plugins.Fullscreen().add_to(m)

gc = plugins.Geocoder(collapsed=True, position='topleft', add_marker=True).add_to(m)

mc = plugins.MeasureControl( position='topleft').add_to(m)


m

In [ ]:
questionnaire[questionnaire['Are you:'].str.contains("who attends games")]